In [10]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset , DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [13]:
data = pd.read_csv("riceClassification.csv")


In [14]:
data.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [16]:
## data preprocessing 
data.dropna(inplace=True)
data.drop(['id'], axis=1, inplace=True)

In [17]:
data.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [19]:
print(data["Class"].unique())

[1 0]


In [21]:
print(data["Class"].value_counts())

Class
1    9985
0    8200
Name: count, dtype: int64


In [22]:
## normalization
original_data = data.copy()

In [23]:
for column in data.columns:
    data[column] = data[column]/data[column].abs().max()
data.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [24]:
X = np.array(data.iloc[:,:-1]) ## taking all rows and column expect the last column
Y = np.array(data.iloc[:,-1])

In [25]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y,test_size = 0.3)

In [26]:
 X_test ,X_val, Y_test,Y_val = train_test_split(X_test,Y_test,test_size = 0.5)

In [30]:
X_train.shape , X_test.shape , X_val.shape

((12729, 10), (2728, 10), (2728, 10))

In [32]:
class dataset(Dataset):
    def __init__(self,X,Y):
        self.X = torch.tensor(X,dtype = torch.float32)
        self.Y = torch.tensor(Y,dtype = torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]
    

In [33]:
training_data = dataset(X_train,Y_train)
validation_data = dataset(X_val,Y_val)
testing_data = dataset(X_test,Y_test)

In [34]:
train_dataloader = DataLoader(training_data,batch_size = 8,shuffle=True)
validation_dataloader = DataLoader(validation_data,batch_size = 8,shuffle=True)
testing_dataloader = DataLoader(testing_data,batch_size = 8,shuffle=True)


In [35]:
HIDDEN_NEURONS = 10
class Mymodel(nn.Module):
    def __init__(self):
        super(Mymodel,self).__init__()
        self.input_layer = nn.Linear(X.shape[1],HIDDEN_NEURONS)
        self.linear = nn.Linear(HIDDEN_NEURONS,1)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,X):
        x = self.input_layer(X)
        x = self.linear(X)
        x = self.sigmoid(X)
        return x
    
model = Mymodel()

In [36]:
summary(model , (X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                   [-1, 10]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
